In [9]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [10]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [11]:
def plot_predictions(test,predicted, title='horizontal_motion[pixel]'):
    plt.plot(test, color='red',label='Real')
    plt.plot(predicted, color='blue',label='Predicted')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel(title)
    plt.legend()
    plt.show()

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

def plot_colored_line(values, X):
    """
    Plots a line graph with the first X values in red and the rest in blue.

    :param values: List of values to plot.
    :param X: Number of initial values to plot in red.
    """
    # Splitting the data
    x_values = range(len(values))
    red_x = x_values[:X]
    red_y = values[:X]
    blue_x = x_values[X-1:]  # Overlap at the point of color change
    blue_y = values[X-1:]

    # Plotting
    plt.plot(red_x, red_y, color='red')
    plt.plot(blue_x, blue_y, color='blue')

    # Adding titles and labels (optional)
    plt.title("Line Plot with Different Color Segments")
    plt.xlabel("Time")
    plt.ylabel("Values")

    # Show plot
    plt.show()

In [12]:
all_variables = ['time[s]',
       'horizontal_motion[deg]', 'vertical_motion[deg]']

dir = './MCI/eye_motion_trace'

In [13]:
all_x_train = []
all_y_train = []

for col in all_variables:
    col_x_train = []
    col_y_train = []

    for i in os.listdir(dir + '/'):
        df = pd.read_csv(dir + '/' + i)
        df = df[[col]]
    
        df_arr = np.array(df)
        divider = int(0.8 * len(df))
        training_set = df_arr[:divider]
        test_set = df_arr[divider:]

        sc = MinMaxScaler(feature_range=(0,1)) #Transform to 0-1 domain
        training_set_scaled = sc.fit_transform(training_set)
        
        for i in range(60,len(training_set_scaled)):
            col_x_train.append(training_set_scaled[i-60:i,0])
            col_y_train.append(training_set_scaled[i,0])

        if(len(col_x_train) >= 5000):
            break

    col_x_train, col_y_train = np.array(col_x_train), np.array(col_y_train)
    all_x_train.append(col_x_train)
    all_y_train.append(col_y_train)

In [14]:
# We don't need this much data
# print(len(all_x_train[0])) --> 294840

In [15]:
print(len(all_x_train[0])) 

7560


In [16]:
all_regressors = []
for i in range(len(all_variables)):
    # The LSTM architecture
    regressor = Sequential()
    # First LSTM layer with Dropout regularisation
    regressor.add(LSTM(units=50, return_sequences=True, input_shape=(60, 1)))
    regressor.add(Dropout(0.2))
    # Second LSTM layer
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.2))
    # Third LSTM layer
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.2))
    # Fourth LSTM layer
    regressor.add(LSTM(units=50))
    regressor.add(Dropout(0.2))
    # The output layer
    regressor.add(Dense(units=1))
    
    # regressor.summary()
    regressor.compile(optimizer='rmsprop',loss='mean_squared_error')
    regressor.fit(all_x_train[i],all_y_train[i],epochs=30,batch_size=64)
    all_regressors.append(regressor)

Epoch 1/30
119/119 [==============================] - 11s 74ms/step - loss: 0.0227
Epoch 2/30
119/119 [==============================] - 9s 78ms/step - loss: 0.0084
Epoch 3/30
119/119 [==============================] - 9s 73ms/step - loss: 0.0063
Epoch 4/30
119/119 [==============================] - 9s 77ms/step - loss: 0.0049
Epoch 5/30
119/119 [==============================] - 9s 79ms/step - loss: 0.0042
Epoch 6/30
119/119 [==============================] - 9s 74ms/step - loss: 0.0036
Epoch 7/30
119/119 [==============================] - 9s 75ms/step - loss: 0.0033
Epoch 8/30
119/119 [==============================] - 9s 77ms/step - loss: 0.0029
Epoch 9/30
119/119 [==============================] - 9s 73ms/step - loss: 0.0028
Epoch 10/30
119/119 [==============================] - 9s 77ms/step - loss: 0.0025
Epoch 11/30
119/119 [==============================] - 9s 75ms/step - loss: 0.0024
Epoch 12/30
119/119 [==============================] - 9s 78ms/step - loss: 0.0023
Epoch 13/30


In [17]:
all_sc = []
for i in range(len(all_y_train)):
    sc = MinMaxScaler(feature_range=(0,1)) #Transform to 0-1 domain
    inputs = sc.fit(all_y_train[i])
    inputs = sc.transform(all_y_train[i])
    all_y_train[i] = inputs
    all_sc.append(sc)

print(all_y_train[i])

[[0.17405765]
 [0.1759054 ]
 [0.17738359]
 ...
 [0.87004219]
 [0.86891702]
 [0.86835443]]


In [18]:
starting_windows = []
for i in range(len(all_variables)):
    cur = all_y_train[i]
    starting_windows.append(cur[len(cur) - 60 : len(cur)])

starting_windows = np.array(starting_windows)
print(starting_windows.shape)

(3, 60, 1)


In [19]:
# X_test = []
# for i in range(60,len(inputs)):
#     X_test.append(inputs[i-60:i,0])
    
# X_test = np.array(X_test)
# X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

# pred = regressor.predict(X_test)
# pred = sc.inverse_transform(pred)

generated_frames = 10000
all_frames = []

for i in range(len(all_variables)):
    cur_frames = list(starting_windows[i])
    for j in range(generated_frames):
        pred = all_regressors[i].predict(np.array(cur_frames[len(cur_frames)-60:len(cur_frames)]))
        cur_frames.append(pred[0])
    
    all_frames.append(np.array(cur_frames))

2/2 [==============================] - 0s 860us/step


KeyboardInterrupt: 

In [20]:
for i in range(len(all_frames)):
    all_frames[i] = all_sc[i].inverse_transform(all_frames[i])

In [21]:
%%capture
testing = list(starting_windows[0])
for i in range(1000):
    pred = regressor.predict(np.array(testing[len(testing)-60:len(testing)]))
    testing.append(pred[0])

In [22]:
testing

[array([0.98438949]),
 array([0.98465623]),
 array([0.98492296]),
 array([0.985177]),
 array([0.98544374]),
 array([0.98571048]),
 array([0.98597721]),
 array([0.98624395]),
 array([0.98651069]),
 array([0.98676472]),
 array([0.98703146]),
 array([0.9872982]),
 array([0.98756494]),
 array([0.98783168]),
 array([0.98809841]),
 array([0.98835245]),
 array([0.98861919]),
 array([0.98888593]),
 array([0.98915266]),
 array([0.9894194]),
 array([0.98968614]),
 array([0.98995288]),
 array([0.99020691]),
 array([0.99047365]),
 array([0.99074039]),
 array([0.99100713]),
 array([0.99127386]),
 array([0.9915279]),
 array([0.99179464]),
 array([0.99206138]),
 array([0.99232811]),
 array([0.99259485]),
 array([0.99286159]),
 array([0.99311562]),
 array([0.99338236]),
 array([0.9936491]),
 array([0.99391584]),
 array([0.99418258]),
 array([0.99444931]),
 array([0.99470335]),
 array([0.99497009]),
 array([0.99523683]),
 array([0.99550356]),
 array([0.9957703]),
 array([0.99603704]),
 array([0.9963037

(1060, 1)
